In [89]:
"""
Data Generation

NEED TO INCLUDE RUNE DATA AND TAG RUNES TO CHAMPION ROLES (i.e FIGHTERS GET CONQ, ASASSINS ELCTROCUTE)
"""

import random
import json

def generate_dummy_match_data(num_matches=100000):
    # Load champion and item data from JSON files
    with open("../data/raw/champion_data/champions.json", 'r') as f:
        champions_data = json.load(f)
        champions = [champion for champion in champions_data["data"].keys()]

    with open("../data/raw/item_data/items.json", 'r') as f:
        items_data = json.load(f)
        items = items_data["data"]

    # Categorize items based on tags and gold value
    damage_items = [item_info["name"] for item_info in items.values() if "Damage" in item_info.get("tags", []) and item_info.get("gold", {}).get("total", 0) > 1500]
    health_items = [item_info["name"] for item_info in items.values() if "Health" in item_info.get("tags", []) and item_info.get("gold", {}).get("total", 0) > 1500]
    spell_damage_items = [item_info["name"] for item_info in items.values() if "SpellDamage" in item_info.get("tags", []) and item_info.get("gold", {}).get("total", 0) > 1500]
    defense_items = [item_info["name"] for item_info in items.values() if ("Armor" in item_info.get("tags", []) or "SpellBlock" in item_info.get("tags", [])) and item_info.get("gold", {}).get("total", 0) > 1500]

    matches = []
    num_wins = num_matches // 2
    num_losses = num_matches - num_wins

    for _ in range(num_wins):
        team = random.choice(["Red", "Blue"])
        champion_name = random.choice(champions)
        champion_info = champions_data["data"][champion_name]["info"]
        attack = champion_info["attack"]
        defense = champion_info["defense"]
        magic = champion_info["magic"]

        # Determine item pool based on champion attributes
        item_pool = []
        if attack >= max(defense, magic):
            item_pool += damage_items
        if defense >= max(attack, magic):
            item_pool += health_items + defense_items
        if magic >= max(attack, defense):
            item_pool += spell_damage_items

        # Ensure at least some items are chosen if the item pool is empty
        if not item_pool:
            item_pool = random.sample(damage_items + health_items + spell_damage_items, k=min(6, len(damage_items + health_items + spell_damage_items)))

        kda = {
            "kills": random.randint(0, 30),
            "deaths": random.randint(0, 10),
            "assists": random.randint(5, 20)
        }
        match = {
            "team": team,
            "summoner_name": f"Player{random.randint(1, 10000)}",
            "champion": champion_name,
            "kda": f"{kda['kills']}/{kda['deaths']}/{kda['assists']}",
            "items": random.sample(item_pool, k=min(6, len(item_pool))),
            "won": True,
            "opponent": random.choice(champions),
        }
        matches.append(match)

    for _ in range(num_losses):
        team = random.choice(["Red", "Blue"])
        champion_name = random.choice(champions)
        champion_info = champions_data["data"][champion_name]["info"]
        attack = champion_info["attack"]
        defense = champion_info["defense"]
        magic = champion_info["magic"]

        # Determine item pool based on champion attributes
        item_pool = []
        if attack >= max(defense, magic):
            item_pool += damage_items
        if defense >= max(attack, magic):
            item_pool += health_items + defense_items
        if magic >= max(attack, defense):
            item_pool += spell_damage_items

        # Ensure at least some items are chosen if the item pool is empty
        if not item_pool:
            item_pool = random.sample(damage_items + health_items + spell_damage_items, k=min(6, len(damage_items + health_items + spell_damage_items)))

        kda = {
            "kills": random.randint(0, 10),
            "deaths": random.randint(5, 20),
            "assists": random.randint(0, 10)
        }
        match = {
            "team": team,
            "summoner_name": f"Player{random.randint(1, 10000)}",
            "champion": champion_name,
            "kda": f"{kda['kills']}/{kda['deaths']}/{kda['assists']}",
            "items": random.sample(item_pool, k=min(6, len(item_pool))),
            "won": False,
            "opponent": random.choice(champions),
        }
        matches.append(match)

    # Shuffle matches to mix wins and losses
    random.shuffle(matches)

    # Save the dummy data to a JSON file
    with open("../data/dummy_match_data.json", "w") as f:
        json.dump(matches, f, indent=4)

    # Generate a formatted output example
    for match in matches[:10]:  # Print a few example matches
        items_str = ", ".join(match["items"])
        print(f"{match['summoner_name']} played {match['champion']} with {match['kda']} and built {items_str}")

    print("\nDummy match data generated and saved.")

# Generate dummy data
generate_dummy_match_data()

Player1158 played Nidalee with 5/10/8 and built Legendary Support Item, Night Harvester, Moonstone Renewer, Cryptbloom, Seraph's Embrace, Dawncore
Player9433 played Poppy with 14/8/12 and built Warmog's Armor, Iceborn Gauntlet, Frozen Heart, Anathema's Chains, Spirit Visage, Hullbreaker
Player288 played Shaco with 7/19/2 and built Yun Tal Wildarrows, Serpent's Fang, Serylda's Grudge, Shield of the Rakkor, Lord Dominik's Regards, Overlord's Bloodmail
Player3941 played Blitzcrank with 22/3/15 and built Stridebreaker, Evenshroud, Locket of the Iron Solari, Edge of Night, Thornmail, Morellonomicon
Player2335 played Lillia with 17/2/16 and built Rabadon's Deathcap, Stormsurge, Echoes of Helia, Imperial Mandate, Moonstone Renewer, Rylai's Crystal Scepter
Player7757 played Ivern with 2/3/15 and built Nashor's Tooth, Horizon Focus, Riftmaker, Staff of Flowing Water, Seraph's Embrace, Liandry's Torment
Player8064 played Kassadin with 13/6/17 and built Night Harvester, Stormsurge, Shadowflame, R

In [90]:
# loading dummy data into dataframe
import pandas as pd
import json

# Load dummy match data into a pandas DataFrame
with open("../data/dummy_match_data.json", "r") as f:
    match_data = json.load(f)

# Convert the list of matches to a DataFrame
df = pd.DataFrame(match_data)

df['champion'].value_counts()


champion
Qiyana     679
Zed        648
Braum      646
Sivir      646
DrMundo    643
          ... 
Renata     544
Galio      543
Yorick     539
Shen       529
Jhin       528
Name: count, Length: 169, dtype: int64

In [93]:
df.head()

,team,summoner_name,champion,kda,items,won,opponent
0,Red,Player1158,Nidalee,5/10/8,"[Legendary Support Item, Night Harvester, Moon...",False,Ashe
1,Red,Player9433,Poppy,14/8/12,"[Warmog's Armor, Iceborn Gauntlet, Frozen Hear...",True,Riven
2,Blue,Player288,Shaco,7/19/2,"[Yun Tal Wildarrows, Serpent's Fang, Serylda's...",False,Jax
3,Red,Player3941,Blitzcrank,22/3/15,"[Stridebreaker, Evenshroud, Locket of the Iron...",True,JarvanIV
4,Blue,Player2335,Lillia,17/2/16,"[Rabadon's Deathcap, Stormsurge, Echoes of Hel...",True,Kaisa


In [63]:
df.dtypes

team             object
summoner_name    object
champion         object
kda              object
items            object
won                bool
opponent         object
dtype: object

# Feature Engineering (Data Preprocessing)

## Functioning Pipeline For Object Columns Named team, summoner_name, champion, kda, items, opponent and boolean named won

In [97]:
from sklearn.preprocessing import OneHotEncoder, StandardScaler, LabelEncoder, MultiLabelBinarizer
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.base import BaseEstimator, TransformerMixin
import numpy as np

# Load dummy match data into a pandas DataFrame
with open("../data/dummy_match_data.json", "r") as f:
    match_data = json.load(f)

# Convert the list of matches to a DataFrame
df = pd.DataFrame(match_data)

# Label encode the 'summoner_name' column
label_encoder = LabelEncoder()
df['summoner_name'] = label_encoder.fit_transform(df['summoner_name'])

# Identifying feature types
numerical_features = ['kills', 'deaths', 'assists', 'summoner_name']
categorical_features = ['team', 'champion', 'opponent']
boolean_features = ['won']

# Custom Transformer for splitting the KDA column
class SplitKDATransformer(BaseEstimator, TransformerMixin):
    def fit(self, X, y=None):
        return self
    
    def transform(self, X, y=None):
        X_copy = X.copy()
        kda_split = X_copy['kda'].str.split('/', expand=True)
        X_copy['kills'] = pd.to_numeric(kda_split[0])
        X_copy['deaths'] = pd.to_numeric(kda_split[1])
        X_copy['assists'] = pd.to_numeric(kda_split[2])
        X_copy.drop(columns=['kda'], inplace=True)
        return X_copy

# Custom Transformer for handling the 'items' column
class ItemsTransformer(BaseEstimator, TransformerMixin):
    def fit(self, X, y=None):
        # Fit a MultiLabelBinarizer to the items column
        self.mlb = MultiLabelBinarizer()
        self.mlb.fit(X['items'])
        return self
    
    def transform(self, X, y=None):
        # Transform the items column
        items_encoded = self.mlb.transform(X['items'])
        items_df = pd.DataFrame(items_encoded, columns=self.mlb.classes_)
        return pd.concat([X.reset_index(drop=True), items_df], axis=1).drop(columns=['items'])

# Ensure the 'won' column is converted to numeric (0/1)
df['won'] = df['won'].astype(int)

# Defining preprocessing steps
numerical_transformer = Pipeline(steps=[
    ('scaler', StandardScaler())
])

categorical_transformer = Pipeline(steps=[
    ('onehot', OneHotEncoder(drop='first', handle_unknown='ignore', sparse_output=False))
])

boolean_transformer = Pipeline(steps=[
    ('onehot', OneHotEncoder(drop='if_binary', sparse_output=False))
])

# Combining preprocessing steps
preprocessor = ColumnTransformer(
    transformers=[
        ('num', numerical_transformer, numerical_features),
        ('cat', categorical_transformer, categorical_features),
        ('bool', boolean_transformer, boolean_features)
    ],
    remainder='passthrough'  # Keep the rest of the columns as they are
)

# Create a pipeline that applies the preprocessor and splits KDA and items
data_pipeline = Pipeline([
    ('split_kda', SplitKDATransformer()),
    ('items', ItemsTransformer()),
    ('preprocessor', preprocessor),
])

# Apply the pipeline to the DataFrame
transformed_data = data_pipeline.fit_transform(df)

# Define new columns based on the transformations
columns = (
    numerical_features +
    list(preprocessor.named_transformers_['cat'].named_steps['onehot'].get_feature_names_out(categorical_features)) +
    list(preprocessor.named_transformers_['bool'].named_steps['onehot'].get_feature_names_out(boolean_features)) +
    list(data_pipeline.named_steps['items'].mlb.classes_)
)

# Convert the transformed data back to a pandas DataFrame
transformed_df = pd.DataFrame(transformed_data, columns=columns)

In [105]:
transformed_df.head()

,kills,deaths,assists,summoner_name,team_Red,champion_Ahri,champion_Akali,champion_Akshan,champion_Alistar,champion_Ambessa,...,Voltaic Cyclosword,Warmog's Armor,Winter's Approach,Wit's End,Wooglet's Witchcap,Wordless Promise,Youmuu's Ghostblade,Yun Tal Wildarrows,Zeke's Convergence,Zhonya's Hourglass
0,-0.597032,0.229378,-0.139319,-1.666606,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.482846,-0.138474,0.594208,1.519419,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,-0.357059,1.884713,-1.239609,-1.004035,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
3,1.442737,-1.058105,1.144353,-0.595126,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.842805,-1.242031,1.327735,-1.213341,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


# Model Creation (Content Based Filtering)